In [1]:
#pip install PySimpleGUI (one time instalation of the package that allows variable selection)

In [2]:
# pandas
import pandas as pd 
# pip install eurostat (one time instalation of the package) 
#TOOL TO IMPORT EUROSTAT DATA
import eurostat
df = eurostat.get_data_df('sbs_sc_sca_r2', flags=False)
df

,nace_r2,indic_sb,size_emp,geo\time,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,B,V11110,0-9,AT,NaN,183.0,192.0,215.0,226.0,230.0,227.0,238.0,238.0,242.0,245.0,244.0,237.0,NaN,NaN,NaN
1,B,V11110,0-9,BA,NaN,101.0,102.0,121.0,129.0,132.0,121.0,118.0,121.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN
2,B,V11110,0-9,BE,NaN,140.0,118.0,173.0,144.0,167.0,149.0,175.0,160.0,227.0,249.0,248.0,NaN,NaN,NaN,NaN
3,B,V11110,0-9,BG,NaN,190.0,211.0,215.0,224.0,240.0,228.0,243.0,252.0,245.0,241.0,230.0,199.0,NaN,NaN,NaN
4,B,V11110,0-9,CH,NaN,82.0,84.0,81.0,85.0,84.0,86.0,89.0,75.0,75.0,79.0,85.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475052,S95,V92100,TOTAL,RS,NaN,4.4,4.3,4.2,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475053,S95,V92100,TOTAL,SE,NaN,1.1,1.1,1.2,1.2,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475054,S95,V92100,TOTAL,SI,NaN,1.5,1.5,1.5,1.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475055,S95,V92100,TOTAL,SK,NaN,1.5,1.4,1.5,1.4,1.4,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#eliminate columns for years previous to 2011
years =[]
for y in range(2005,2011): 
    years.append(y)
df=df.drop(years, axis=1)#quitamos todas las columnas de añosque no nos interesan
#rename geo time column to be easier to handle because of the name
df.rename(columns={"geo\\time": "geo"}, inplace=True)
#We can get the unique values of the different variables

In [4]:
#define selection query box

import PySimpleGUI as sg
def selection(option):
    #sg.theme('DarkAmber')   # Add a touch of color
    #select the variable we want to calculate
    options = option

    # All the stuff inside your window.
    layout = [ 
            [sg.Text('Select one->'), sg.Listbox(options,select_mode=sg.LISTBOX_SELECT_MODE_SINGLE,size=(20,len(options)))],
            [sg.Button('Ok'), sg.Button('Cancel')]
        ]

    # Create the Window
    window = sg.Window('Make your choice', layout)

    # Event Loop to process "events" and get the "values" of the input
    while True:
        event, values = window.read()
        print( f"event={event}" )
        if event is None or event == 'Ok' or event == 'Cancel': # if user closes window or clicks cancel
            break
        
    # close  the window        
    window.close()

    if event == "Cancel":
        print( "You cancelled" )
    else:
        print('You entered ', values[0])
        sg.popup( f"You selected {values[0]}" )        
    return values

In [5]:
#From the total database we are going to select the variable of interest and the time period to calculate de rations

#selection of the variable
list_of_variables = pd.read_excel("variabledescription.xlsx")
var=selection(df.indic_sb.unique())[0]
condition=list_of_variables[list_of_variables["variable"]==var[0]]
variable_of_interest=condition.iloc[0]['description']
print(variable_of_interest)
print(" ")
#var=values #with [] we convert variable to a list to apply "isin"
print(var)


#sedlect the size
#size_sel=selection(df.size_emp.unique())[0] #with [] we convert variable to a list to apply "isin"
#print(size_sel)



#select de year frame
yearsloop =[]
for y in range(2011,2021): 
    yearsloop.append(y)
year_start=selection(yearsloop)[0] #with [] we convert variable to a list to apply "isin"
year_end=selection(yearsloop)[0]
print(year_start)
print(year_end)

#we select ['V12150'] Value added at factor cost - million euro
# and 2011 to 2019

event=Ok
You entered  ['V12150']
 Value added at factor cost - million euro
 
['V12150']
event=Ok
You entered  [2011]
event=Ok
You entered  [2019]
[2011]
[2019]


In [6]:
#list of countries available
countries=df.geo.unique()
print("Countries", countries)
print(" ")
#list of variables available
variables=df.indic_sb.unique()
print("Variables", variables)
print(" ")
#see size groups
size=df.size_emp.unique()
print("Size",size)
print(" ")
#load weights database
weights_target = pd.read_excel("ecosystemweightsextended.xlsx")
#check nace groups in weights
naceweights=weights_target.NACE_R2.unique()
nace_groups={'C10-C12','C13-C15', 'E37-E39','J59_J60','J62_J63','M74_M75', 'N77_N78', 'D35'}
naceweights=set(naceweights)-set(nace_groups)
#check nace groups in the database
nacedatabase=df.nace_r2.unique()
#look for the differences to eliminate nace codes
nacedrop=set(nacedatabase)-set(naceweights)
print("Nace weights",naceweights)
print(" ")
print("Nace database",nacedatabase)
print(" ")
print("Nace to drop",nacedrop)
print(" ")
#countries to eliminate
countries_not_selected= ['UK', 'TR', 'AL','IS','LI','NO','CH','UK','ME','MK','AL','RS','TR','BA','EU28','EU15','EA','EA19','EA12', 'EU27_2007']
print(countries_not_selected)
print(" ")
#select the size we want to evaluate

Countries ['AT' 'BA' 'BE' 'BG' 'CH' 'CY' 'CZ' 'DE' 'DK' 'EE' 'EL' 'ES' 'EU27_2007'
 'EU27_2020' 'EU28' 'FI' 'FR' 'HR' 'HU' 'IE' 'IS' 'IT' 'LT' 'LU' 'LV' 'MK'
 'MT' 'NL' 'NO' 'PL' 'PT' 'RO' 'RS' 'SE' 'SI' 'SK' 'UK' 'AL' 'TR']
 
Variables ['V11110' 'V12110' 'V12120' 'V12150' 'V12170' 'V13110' 'V13310' 'V13320'
 'V13330' 'V16110' 'V16120' 'V16130' 'V16150' 'V91100' 'V91110' 'V91120'
 'V91130' 'V91150' 'V91170' 'V91210' 'V91230' 'V91275' 'V91290' 'V91310'
 'V92100' 'V92110' 'V92111' 'V92112' 'V94210' 'V94240' 'V94270' 'V94310']
 
Size ['0-9' '10-19' '20-49' '50-249' 'GE250' 'TOTAL']
 
Nace weights {'N78', 'J60', 'P', 'G46', 'C19', 'T', 'C31', 'C17', 'H51', 'C15', 'C23', 'H50', 'G47', 'F', 'P85', 'C13', 'E39', 'C26', 'N81', 'C24', 'C10', 'C16', 'Q87_Q88', 'J61', 'M71', 'S94', 'C11', 'C20', 'C28', 'E38', 'J63', 'N77', 'C14', 'J62', 'M72', 'C33', 'C18', 'J59', 'N79', 'C25', 'I', 'H52', 'M69_M70', 'C22', 'R90-R92', 'M70', 'R93', 'M73', 'E37', 'C21', 'D', 'C32', 'C29', 'J58', 'G45', 'Q86', 'C12

In [7]:
import numpy as np
#we create an empty dataframe to store the total results
results_compiled_tot = pd.DataFrame()
#we create an empty dataframe to store the ratio results
ratio_compiled_tot = pd.DataFrame()

#we create a list with the different size classes in the database
size_classes= df.size_emp.nunique()
#size= [set(df.size_emp.unique())]
size= df.size_emp.unique()

# columns of countries to be eliminated
cols = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI','FR', 'HR', 'HU', 'IE', 'IT', 'LT','LU', 'LV', 'MT','NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']

In [8]:
#loop for the years to calculate the value of the variable of interest (e.g. value added) for each size group
for i in range (year_start[0],year_end[0]+1):
    year=i
    #we create two dataframes to store the partial calculations for each year One for the absolute values and another for the rations
    results_compiled = pd.DataFrame()
    ratio_compiled = pd.DataFrame()
    #loop for the classes
    for j in range (0,size_classes):
        #size_sel is the class selected in each step of the loop
        size_sel=[size[j]]
        #we select the rows that have the right size, nace codes, variables and countries ~indicates not in the group
        df_selection=df[df.size_emp.isin(size_sel)&~df.nace_r2.isin(nacedrop)&df.indic_sb.isin(var)&~df.geo.isin(countries_not_selected)]
        #print(size_sel)
        #create a dataframe for the results by transposing the values to columns
        #print(df_selection)
        df_target = pd.pivot_table(df_selection, values=year, index=['nace_r2'],columns=["geo"], aggfunc=np.sum, fill_value=0)#reorde
        #print(df_target)
        df_target['sum'] = df_target[cols].sum(axis=1)#sums the columns of the countries to "sum"
        df_target=df_target.drop(cols, axis=1)#we eliminate the columns with the countries
        #now we can locate columns with 0 and make the substitution with the column "sum" for missing data in the totals
        df_target.loc[df_target['EU27_2020']==0,'EU27_2020'] = df_target['sum']
        moddf_target=df_target.drop('sum', axis=1) #we drop the column "sum"
        weights_target = pd.read_excel("ecosystemweightsextended.xlsx") # upload the ecosystem weights
        weights_target.set_index('NACE_R2', inplace=True)#we use nace as index in the ecosystem weights
        #print(moddf_target)
        #print(" ")
        #index the nace codes for rows
        pd.set_option('display.float_format', lambda x: '%.4f' % x)#this sets decimals to 4 digits
        column_name=str(variable_of_interest)+str(year)+str(size_sel)#we name the colums with the variable name+year+size
        weights_target[variable_of_interest]=weights_target.index.map(moddf_target['EU27_2020'])#look in df_target for the nace and copies the value in EU27_2020 in the corresponding nace row in weights_target
        #we create the table for results using the weights target structure
        results=weights_target.copy()
        
        for i in range(0,14):#we iterate through the rows to multiply ecosystem weights by the value of the variable of interest (stored in weights target)
            results.iloc[:, i]=weights_target.iloc[:, i]*weights_target[variable_of_interest]#we multiply the total value added by the weight for each nace
        #print(results)
        #print(" ")
        results.loc['total'] = results.select_dtypes(pd.np.number).sum() #we add the values of each colum in a row call total
        #print(results)
        results = results.drop(variable_of_interest, axis=1)
        results_compiled[column_name]=results.loc['total'] #results compiled is storing the totals for a iteration 
        #ratio_compiled[column_name]=results.loc['total']    
        #sum of the totals
        #results_compiled.loc['total'] = results_compiled.select_dtypes(pd.np.number).sum() 
    #ratio_compiled_tot = pd.concat([results_compiled_tot,results_compiled], axis=1)
    #sum_ratio = pd.DataFrame() #creates 
    for j in range (0,6):#iterates through classes to calculate the ratios
        ratio_compiled[j]=results_compiled.iloc[:,j]/results_compiled.iloc[:,5]
    ratio_compiled_tot = pd.concat([ratio_compiled_tot,ratio_compiled], axis=1)#adds ratio_compiled for each class
    results_compiled_tot = pd.concat([results_compiled_tot,results_compiled], axis=1)#results_compiled_total adds results_compiled for each class   
results_compiled_tot 


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,Value added at factor cost - million euro2011['0-9'],Value added at factor cost - million euro2011['10-19'],Value added at factor cost - million euro2011['20-49'],Value added at factor cost - million euro2011['50-249'],Value added at factor cost - million euro2011['GE250'],Value added at factor cost - million euro2011['TOTAL'],Value added at factor cost - million euro2012['0-9'],Value added at factor cost - million euro2012['10-19'],Value added at factor cost - million euro2012['20-49'],Value added at factor cost - million euro2012['50-249'],...,Value added at factor cost - million euro2018['20-49'],Value added at factor cost - million euro2018['50-249'],Value added at factor cost - million euro2018['GE250'],Value added at factor cost - million euro2018['TOTAL'],Value added at factor cost - million euro2019['0-9'],Value added at factor cost - million euro2019['10-19'],Value added at factor cost - million euro2019['20-49'],Value added at factor cost - million euro2019['50-249'],Value added at factor cost - million euro2019['GE250'],Value added at factor cost - million euro2019['TOTAL']
Aerospace & Defence,17609.3729,9810.4318,14419.2114,36306.5363,105746.7153,184455.1412,16025.9695,9720.3395,14975.1588,35029.6561,...,17201.8132,38710.1539,149994.8515,233879.4976,18294.9509,10410.9098,17577.1941,40401.7364,156964.9827,242755.6571
Agri-food,38898.2703,18242.9381,25902.5086,62767.1413,111693.9447,258529.0229,35376.7470,18200.8841,25793.7849,62538.9419,...,29631.5086,66425.2972,177306.6782,328565.5010,35808.5602,19547.9703,30387.4059,67380.4812,186740.5053,340760.9621
Construction,266027.9797,93502.0689,109545.7356,147523.2565,191555.2706,810832.3815,252757.7233,95675.3075,110285.2268,145661.1672,...,134544.3042,177589.4406,295411.1230,1020226.8117,312792.9388,124961.1022,143958.8741,188890.1038,314407.1255,1082450.0612
Cultural and Creative Industries,59873.3668,20642.1419,27819.4290,49213.3580,66175.5828,224559.8445,60852.9997,19700.4026,27148.0804,48260.0053,...,26048.7861,47472.5376,94255.5489,253923.1233,64696.0338,21272.5980,27683.8873,48679.0744,96108.4074,260938.5842
Digital,50846.9941,20609.7516,31775.4418,67137.6181,233303.4121,411304.9952,51269.5512,21210.2880,33766.1517,67534.1646,...,39627.1234,84668.8641,292214.8120,513532.1609,66684.2054,25341.8283,41269.5316,91133.6144,320437.9517,546614.0301
Electronics,6518.4575,5230.9653,7916.6859,22127.2558,54100.3956,95476.7787,6420.1224,4987.9319,8133.3671,21310.2801,...,8977.3548,23559.5397,78099.1221,122226.4443,7331.5003,4986.8372,8972.5423,24198.8088,81786.9520,126579.3691
Energy - Renewables,9333.0905,2959.8847,5348.7952,15151.6017,64632.3546,97727.0729,10125.2948,2853.3395,5445.3791,15167.0542,...,6142.4694,15378.3139,76775.5209,113923.4885,12830.6341,3740.8906,6316.0743,15905.2709,79815.4179,118760.2198
Energy Intensive Industries,29959.2148,21628.3768,36141.3132,105057.4534,230075.1769,423980.3406,28730.8627,19080.5913,34476.1597,100438.2878,...,36870.6357,109964.9461,344600.4968,530259.5523,31126.4000,20242.5096,37306.8380,111436.4616,335228.4652,546536.6440
Health,26408.1226,11564.6678,13170.5933,30983.9424,93826.6825,188886.2683,26374.6651,10507.6828,12502.7160,31356.6806,...,15021.4535,35883.8064,172673.5105,262489.1428,28999.2125,11308.1237,15701.9022,34811.5396,184255.5600,277045.0827
Mobility - Transport - Automotive,94413.1379,42115.5006,61552.5668,106970.5901,279778.1480,585254.5499,90617.6339,41111.8578,58367.1117,102799.4780,...,63223.8877,117445.6490,431916.0031,754980.3077,98378.4602,46249.6664,65961.2542,121544.7948,448544.2365,782349.3005


In [9]:
#based on the total values we can calculate the ratio for each size group by dividing the value for the group and year over total value for the group e.g.  value['0-9']/value ['total']
ratio_compiled_tot.columns=results_compiled_tot.columns #give the same name to the columns in the ratio
ratio_compiled_tot

,Value added at factor cost - million euro2011['0-9'],Value added at factor cost - million euro2011['10-19'],Value added at factor cost - million euro2011['20-49'],Value added at factor cost - million euro2011['50-249'],Value added at factor cost - million euro2011['GE250'],Value added at factor cost - million euro2011['TOTAL'],Value added at factor cost - million euro2012['0-9'],Value added at factor cost - million euro2012['10-19'],Value added at factor cost - million euro2012['20-49'],Value added at factor cost - million euro2012['50-249'],...,Value added at factor cost - million euro2018['20-49'],Value added at factor cost - million euro2018['50-249'],Value added at factor cost - million euro2018['GE250'],Value added at factor cost - million euro2018['TOTAL'],Value added at factor cost - million euro2019['0-9'],Value added at factor cost - million euro2019['10-19'],Value added at factor cost - million euro2019['20-49'],Value added at factor cost - million euro2019['50-249'],Value added at factor cost - million euro2019['GE250'],Value added at factor cost - million euro2019['TOTAL']
Aerospace & Defence,0.0955,0.0532,0.0782,0.1968,0.5733,1.0000,0.0858,0.0520,0.0802,0.1876,...,0.0735,0.1655,0.6413,1.0000,0.0754,0.0429,0.0724,0.1664,0.6466,1.0000
Agri-food,0.1505,0.0706,0.1002,0.2428,0.4320,1.0000,0.1365,0.0702,0.0995,0.2413,...,0.0902,0.2022,0.5396,1.0000,0.1051,0.0574,0.0892,0.1977,0.5480,1.0000
Construction,0.3281,0.1153,0.1351,0.1819,0.2362,1.0000,0.3176,0.1202,0.1386,0.1830,...,0.1319,0.1741,0.2896,1.0000,0.2890,0.1154,0.1330,0.1745,0.2905,1.0000
Cultural and Creative Industries,0.2666,0.0919,0.1239,0.2192,0.2947,1.0000,0.2643,0.0856,0.1179,0.2096,...,0.1026,0.1870,0.3712,1.0000,0.2479,0.0815,0.1061,0.1866,0.3683,1.0000
Digital,0.1236,0.0501,0.0773,0.1632,0.5672,1.0000,0.1219,0.0504,0.0803,0.1606,...,0.0772,0.1649,0.5690,1.0000,0.1220,0.0464,0.0755,0.1667,0.5862,1.0000
Electronics,0.0683,0.0548,0.0829,0.2318,0.5666,1.0000,0.0656,0.0509,0.0831,0.2176,...,0.0734,0.1928,0.6390,1.0000,0.0579,0.0394,0.0709,0.1912,0.6461,1.0000
Energy - Renewables,0.0955,0.0303,0.0547,0.1550,0.6614,1.0000,0.1001,0.0282,0.0538,0.1499,...,0.0539,0.1350,0.6739,1.0000,0.1080,0.0315,0.0532,0.1339,0.6721,1.0000
Energy Intensive Industries,0.0707,0.0510,0.0852,0.2478,0.5427,1.0000,0.0709,0.0471,0.0851,0.2480,...,0.0695,0.2074,0.6499,1.0000,0.0570,0.0370,0.0683,0.2039,0.6134,1.0000
Health,0.1398,0.0612,0.0697,0.1640,0.4967,1.0000,0.1409,0.0561,0.0668,0.1675,...,0.0572,0.1367,0.6578,1.0000,0.1047,0.0408,0.0567,0.1257,0.6651,1.0000
Mobility - Transport - Automotive,0.1613,0.0720,0.1052,0.1828,0.4780,1.0000,0.1586,0.0719,0.1021,0.1799,...,0.0837,0.1556,0.5721,1.0000,0.1257,0.0591,0.0843,0.1554,0.5733,1.0000


In [10]:
#save to file the calculate ratios
fileratio="ratioSMEs-"+ variable_of_interest
filename = "%s.xlsx" % fileratio
ratio_compiled_tot.to_excel(filename)

In [11]:
results_compiled_tot.loc['total'] = results_compiled_tot.select_dtypes(pd.np.number).sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


In [12]:
results_compiled_tot

,Value added at factor cost - million euro2011['0-9'],Value added at factor cost - million euro2011['10-19'],Value added at factor cost - million euro2011['20-49'],Value added at factor cost - million euro2011['50-249'],Value added at factor cost - million euro2011['GE250'],Value added at factor cost - million euro2011['TOTAL'],Value added at factor cost - million euro2012['0-9'],Value added at factor cost - million euro2012['10-19'],Value added at factor cost - million euro2012['20-49'],Value added at factor cost - million euro2012['50-249'],...,Value added at factor cost - million euro2018['20-49'],Value added at factor cost - million euro2018['50-249'],Value added at factor cost - million euro2018['GE250'],Value added at factor cost - million euro2018['TOTAL'],Value added at factor cost - million euro2019['0-9'],Value added at factor cost - million euro2019['10-19'],Value added at factor cost - million euro2019['20-49'],Value added at factor cost - million euro2019['50-249'],Value added at factor cost - million euro2019['GE250'],Value added at factor cost - million euro2019['TOTAL']
Aerospace & Defence,17609.3729,9810.4318,14419.2114,36306.5363,105746.7153,184455.1412,16025.9695,9720.3395,14975.1588,35029.6561,...,17201.8132,38710.1539,149994.8515,233879.4976,18294.9509,10410.9098,17577.1941,40401.7364,156964.9827,242755.6571
Agri-food,38898.2703,18242.9381,25902.5086,62767.1413,111693.9447,258529.0229,35376.7470,18200.8841,25793.7849,62538.9419,...,29631.5086,66425.2972,177306.6782,328565.5010,35808.5602,19547.9703,30387.4059,67380.4812,186740.5053,340760.9621
Construction,266027.9797,93502.0689,109545.7356,147523.2565,191555.2706,810832.3815,252757.7233,95675.3075,110285.2268,145661.1672,...,134544.3042,177589.4406,295411.1230,1020226.8117,312792.9388,124961.1022,143958.8741,188890.1038,314407.1255,1082450.0612
Cultural and Creative Industries,59873.3668,20642.1419,27819.4290,49213.3580,66175.5828,224559.8445,60852.9997,19700.4026,27148.0804,48260.0053,...,26048.7861,47472.5376,94255.5489,253923.1233,64696.0338,21272.5980,27683.8873,48679.0744,96108.4074,260938.5842
Digital,50846.9941,20609.7516,31775.4418,67137.6181,233303.4121,411304.9952,51269.5512,21210.2880,33766.1517,67534.1646,...,39627.1234,84668.8641,292214.8120,513532.1609,66684.2054,25341.8283,41269.5316,91133.6144,320437.9517,546614.0301
Electronics,6518.4575,5230.9653,7916.6859,22127.2558,54100.3956,95476.7787,6420.1224,4987.9319,8133.3671,21310.2801,...,8977.3548,23559.5397,78099.1221,122226.4443,7331.5003,4986.8372,8972.5423,24198.8088,81786.9520,126579.3691
Energy - Renewables,9333.0905,2959.8847,5348.7952,15151.6017,64632.3546,97727.0729,10125.2948,2853.3395,5445.3791,15167.0542,...,6142.4694,15378.3139,76775.5209,113923.4885,12830.6341,3740.8906,6316.0743,15905.2709,79815.4179,118760.2198
Energy Intensive Industries,29959.2148,21628.3768,36141.3132,105057.4534,230075.1769,423980.3406,28730.8627,19080.5913,34476.1597,100438.2878,...,36870.6357,109964.9461,344600.4968,530259.5523,31126.4000,20242.5096,37306.8380,111436.4616,335228.4652,546536.6440
Health,26408.1226,11564.6678,13170.5933,30983.9424,93826.6825,188886.2683,26374.6651,10507.6828,12502.7160,31356.6806,...,15021.4535,35883.8064,172673.5105,262489.1428,28999.2125,11308.1237,15701.9022,34811.5396,184255.5600,277045.0827
Mobility - Transport - Automotive,94413.1379,42115.5006,61552.5668,106970.5901,279778.1480,585254.5499,90617.6339,41111.8578,58367.1117,102799.4780,...,63223.8877,117445.6490,431916.0031,754980.3077,98378.4602,46249.6664,65961.2542,121544.7948,448544.2365,782349.3005


In [13]:
#save to file
filename = "%s.xlsx" % variable_of_interest
results_compiled_tot.to_excel(filename)